In [1]:
import numpy as np
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import RBF
import os
from scipy import sparse
from tqdm import tnrange, tqdm_notebook
from scipy.ndimage import gaussian_filter1d

# Gaussian Smooth Neural

In [2]:
def gauss_smooth_neural(neural_file, sigma, radius, save_dir):
    neural = np.load(neural_file)
    
    for n in tnrange(neural.shape[1]//50):
        neural[:, n*50 : (n+1)*50] = gaussian_filter1d(neural[:,n*50 : (n+1)*50],
                                                     axis=1, sigma=sigma, mode="constant",
                                                       cval=0.0, truncate=radius)
    
    np.save(os.path.join(save_dir, "smooth_lin_50_test_neural.npy"), neural)   

In [3]:
neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_lin_50_test_neural.npy"
sigma=0.5
radius=2
save_dir = "/ssd/joon/2017_11_29_ns/yass/neural/"

gauss_smooth_neural(neural_file, sigma, radius, save_dir)

# Training/Validation for Kernel Ridge

In [2]:
def kernel_reg(train_images_file, valid_images_file, train_neural_file,
                     valid_neural_file, pixel_file, unit_dim, save_dir):
    train_images = np.load(train_images_file)
    valid_images = np.load(valid_images_file)
    train_neural = np.load(train_neural_file)
    valid_neural = np.load(valid_neural_file)
    
    pixel_units = np.load(pixel_file)
    
    image_no = train_images.shape[0]
    pix_no = train_images.shape[1]
    feat_no = train_neural.shape[1]
    
    # FOR SMOOTH# 
    param_grid = {"alpha": np.logspace(-2.5,2.5,5).tolist(),
                  "kernel": [RBF(l) for l in np.logspace(-2, 3,5)]}
   
    kr = GridSearchCV(KernelRidge(), param_grid=param_grid, n_jobs=-1, cv=3)
    
    corr_sum = 0
    decoded = np.empty((valid_images.shape[0], pix_no))
    
    for i in tnrange(pix_no):
        train_pix_img = train_images[:,i]
        unit_ids = pixel_units[i,:unit_dim]
        neural_ids = np.empty((unit_dim*50))
        for j in range(unit_dim):
            unit = unit_ids[j]
            neural_ids[j*50:(j+1)*50] = np.arange(unit*50,(unit+1)*50,1)
        neural_ids = neural_ids.astype(int)
        
        train_pix_neural = train_neural[:, neural_ids]
        valid_pix_neural = valid_neural[:, neural_ids]
        
        valid_pix_img = valid_images[:,i]
        
        kr.fit(train_pix_neural, train_pix_img)
        print(kr.best_params_)
        predict = kr.predict(valid_pix_neural)
        decoded[:,i] = predict
        
        corr = np.corrcoef(valid_pix_img.reshape((-1,)), predict.reshape((-1,)))[0][1]
        print(corr)
        corr_sum += corr
        
        if i%500 == 499:
            np.save(os.path.join(save_dir, "yass_0_to_"+str(i)+"_kern10_hp_decoded.npy"), decoded[:,:i+1])
        
    print("Pix "+str(i), corr_sum/pix_no)
    
    np.save(os.path.join(save_dir, "yass_FULL_kern10_hp_decoded.npy"), decoded)

In [ ]:
train_images_file = "/ssd/joon/2017_11_29_ns/images/hp_train_images.npy"
train_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/smooth_lin_50_train_neural.npy"
valid_images_file = "/ssd/joon/2017_11_29_ns/images/hp_test_images.npy"
valid_neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/smooth_lin_50_test_neural.npy"
pixel_file = "/ssd/joon/2017_11_29_ns/yass/yass_l1_pixel_units.npy"
save_dir = "/ssd/joon/2017_11_29_ns/yass/kern_partial/"
unit_dim = 10




kernel_reg(train_images_file, valid_images_file, train_neural_file, valid_neural_file,
                 pixel_file, unit_dim, save_dir)

{'alpha': 0.0031622776601683794, 'kernel': RBF(length_scale=1e+03)}
0.3220115247712431
{'alpha': 0.0031622776601683794, 'kernel': RBF(length_scale=1e+03)}
0.31447876051107815
{'alpha': 0.0031622776601683794, 'kernel': RBF(length_scale=1e+03)}
0.4183999795549337
{'alpha': 0.0031622776601683794, 'kernel': RBF(length_scale=1e+03)}
0.3664740237817178
{'alpha': 1.0, 'kernel': RBF(length_scale=3.16)}
0.2755376505813489
{'alpha': 1.0, 'kernel': RBF(length_scale=3.16)}
0.4163450295195238
{'alpha': 1.0, 'kernel': RBF(length_scale=56.2)}
0.297594299660028
{'alpha': 1.0, 'kernel': RBF(length_scale=3.16)}
0.10128236256499076
{'alpha': 1.0, 'kernel': RBF(length_scale=3.16)}
0.013410244920753234
{'alpha': 0.0031622776601683794, 'kernel': RBF(length_scale=1e+03)}
0.12036146572987369
{'alpha': 0.0031622776601683794, 'kernel': RBF(length_scale=1e+03)}
0.265159193107245
{'alpha': 0.0031622776601683794, 'kernel': RBF(length_scale=1e+03)}
0.356421693195756
{'alpha': 0.05623413251903491, 'kernel': RBF(leng

In [6]:
print(np.logspace(-1,1.2,6))

[ 0.1         0.27542287  0.75857758  2.08929613  5.75439937 15.84893192]


In [6]:
print(np.logspace(0.7, 2.2,6))

[  5.01187234  10.          19.95262315  39.81071706  79.43282347
 158.48931925]
